# 02-3 AI·LLM·에이전트 활용 실습
## Rule-based Planner + LLM Reasoner 혼합 구조

### 학습목표
- 전통적 AI / LLM / AI 에이전트 비교
- 행동 수행 차이 이해
- Hybrid 구조 이해

## 들어가며
회의록 요약과 후속 작업 자동화는 에이전트 문제이다.

In [ ]:
import json, re
from transformers import pipeline

MODEL_NAME = "google/flan-t5-base"
llm = pipeline("text2text-generation", model=MODEL_NAME, max_new_tokens=256)

def llm_generate(prompt: str) -> str:
    out = llm(prompt, do_sample=False)
    return out[0]["generated_text"].strip()

print("LLM ready:", MODEL_NAME)

In [ ]:
def make_summary_todos(meeting_text: str) -> str:
    prompt = f"""
다음 회의록을 한국어로 4줄 이내 요약하고,
To-do를 3개 bullet로 작성하라.

회의록:
{meeting_text}
"""
    result = llm_generate(prompt)
    if not result or len(result.strip()) < 20:
        return "(요약 생성 실패)\n\n- (To-do) 후속 조치 1\n- (To-do) 후속 조치 2\n- (To-do) 후속 조치 3"
    return result

In [ ]:
def calculator(expression: str) -> float:
    return eval(expression)

def save_text(filepath: str, content: str):
    import os
    os.makedirs(os.path.dirname(filepath), exist_ok=True)
    with open(filepath, "w", encoding="utf-8") as f:
        f.write(content)

In [ ]:
def planner():
    return {
        "steps": [
            {"tool": "make_summary_todos"},
            {"tool": "calculator", "expression": "30000000 * 0.15"},
            {"tool": "save_text", "filepath": "output/agent_result.txt"}
        ],
        "final_message": "회의록 처리를 완료했습니다."
    }

In [ ]:
def run_agent(meeting_text: str):
    plan = planner()
    summary = make_summary_todos(meeting_text)
    calc_result = calculator("30000000 * 0.15")
    final_content = f"{summary}\n\n[계산] 3천만 원의 15% = {calc_result} 원"
    save_text("output/agent_result.txt", final_content)
    print("FINAL:", plan["final_message"])
    print("\n--- 저장된 파일 ---")
    print(final_content)

In [ ]:
MEETING_TEXT = """
- 마케팅 캠페인 일정 논의
- 예산은 3천만 원
- SNS 홍보 강화 필요
"""

run_agent(MEETING_TEXT)

## 학습정리
Rule-based Planner + LLM Reasoner 구조가 가장 안정적이다.